In [7]:
from query.datasets.prelude import *
import tempfile

In [4]:
v = Video.objects.get(path='tvnews/videos/MSNBCW_20160505_230000_Hardball_With_Chris_Matthews.mp4')

https://storage.googleapis.com/esper/tvnews/videos/MSNBCW_20160505_230000_Hardball_With_Chris_Matthews.mp4?GoogleAccessId=esper-111@visualdb-1046.iam.gserviceaccount.com&Expires=1519508620&Signature=sQTJKKvp62XjYfKpWskHI%2BdYttRFuBGT1iXsFT77%2FSLvVN374cOFXniqMDXtC%2Br9vzu01gpikf3r2mr580kjToPcDpkCfWT5%2BQYu%2F5htEHmaKd26oujmY9JcYO6aa%2FRFPh8KtvhKFQIMQJ%2BbKFmfPpTIKAk%2B%2Fw3%2FKv2z3E9C5ATx%2FLCKDTMkCz9qLayXzUWEQevhIhebJMndZOWaayt9G%2FgQB3zxaee3ayZrbFzGCV51Q%2F4tuilOqwuZEr%2BlyI6SpK8Dd1Nmfzs4HPDWbUDDHSNL0XMueHEsqt6AKYfbr2wUxZJ%2FgUAwEmdcdx0oqvQdeV7HTRDMBMWOiR8uVMw88g%3D%3D


In [16]:
def extract_audio(video, output_path=None, ext='wav', segment=None):
    if output_path is None:
        output_path = tempfile.NamedTemporaryFile(suffix='.{}'.format(ext), delete=False).name
        
    def fmt_time(t):
        return '{:02d}:{:02d}:{:02d}.{:03d}'.format(
            int(t / 3600), int(t / 60 % 60), int(t % 60), int(t * 1000 % 1000))    
    
    if segment is not None:
        (start, end) = segment
        start_str = '-ss {}'.format(fmt_time(start))
        end_str = '-t {}'.format(fmt_time(end - start))
    else:
        start_str = ''
        end_str = ''
        
    sp.check_call('ffmpeg -y {} -i "{}" {} {}'.format(start_str, video.url(), end_str, output_path), shell=True)
    return output_path

In [23]:
def parse_segmentation(lines):
    seg = []
    for line in lines:
        if line[:2] == ';;':
            continue
            
        [start, end, gender] = line.split(' ')[2:5]
        seg.append({
            'start': float(start)/100,
            'end': float(end)/100,
            'gender': gender
        })
        
    return seg
        
def segment_audio(video, segment=None):
    with Timer('Extracting audio'):
        audio_path = extract_audio(video, segment=segment)
        
    with Timer('Segmenting audio'):
        seg_path = tempfile.NamedTemporaryFile(delete=False).name
        sp.check_call(
            'java -jar /app/deps/LIUM/LIUM_SpkDiarization-4.2.jar --fInputMask={} --sOutputMask={} _'
            .format(audio_path, seg_path),
            shell=True)
    
    with open(seg_path) as f:
        seg_lines = [s.strip() for s in f.readlines()]
    seg = parse_segmentation(seg_lines)
    
    os.remove(audio_path)
    os.remove(seg_path)
    
    return seg

In [25]:
seg = segment_audio(v)
print(seg)

D 18-02-24 14:01:12 prelude.py:249] -- START: Extracting audio
D 18-02-24 14:01:24 prelude.py:256] -- END: Extracting audio -- 00:00:12
D 18-02-24 14:01:24 prelude.py:249] -- START: Segmenting audio
D 18-02-24 14:03:55 prelude.py:256] -- END: Segmenting audio -- 00:02:30


[{'start': 0.0, 'end': 7.24, 'gender': 'M'}, {'start': 7.5, 'end': 5.78, 'gender': 'M'}, {'start': 13.88, 'end': 2.89, 'gender': 'M'}, {'start': 533.42, 'end': 5.49, 'gender': 'M'}, {'start': 644.09, 'end': 1.51, 'gender': 'M'}, {'start': 690.37, 'end': 9.93, 'gender': 'M'}, {'start': 700.44, 'end': 3.2, 'gender': 'M'}, {'start': 1210.98, 'end': 11.26, 'gender': 'M'}, {'start': 3615.89, 'end': 4.13, 'gender': 'M'}, {'start': 18.52, 'end': 3.21, 'gender': 'M'}, {'start': 31.98, 'end': 18.9, 'gender': 'M'}, {'start': 172.8, 'end': 18.72, 'gender': 'M'}, {'start': 249.24, 'end': 14.67, 'gender': 'M'}, {'start': 316.37, 'end': 8.08, 'gender': 'M'}, {'start': 324.45, 'end': 12.26, 'gender': 'M'}, {'start': 336.71, 'end': 4.02, 'gender': 'M'}, {'start': 372.05, 'end': 12.87, 'gender': 'M'}, {'start': 429.85, 'end': 16.23, 'gender': 'M'}, {'start': 446.08, 'end': 9.16, 'gender': 'M'}, {'start': 455.24, 'end': 3.37, 'gender': 'M'}, {'start': 458.61, 'end': 10.89, 'gender': 'M'}, {'start': 469.

In [27]:
json.dump(seg, open('/app/gender_seg.json', 'wb'))